- Authors: Marc Brooks, Jack McCarthy, Michael Sarkis, Itamar Barak
- NetID: mgb45, Jwm70 , ms939, imb12

### 1.

#### Part 1

$$\begin{aligned}
\mathbb{E}{\lvert\lvert k^{-1/2}\Omega \textbf{x}\rvert\rvert}^2 &= {\lvert\lvert\textbf{x}\rvert\rvert}^2\\
&= \frac{1}{k}\mathbb{E}\left[\textbf{x}^T\Omega^T\Omega\textbf{x}\right]\\
&= tr\left(\frac{1}{k}\mathbb{E}\left[\textbf{x}^T\Omega^T\Omega\textbf{x}\right]\right)\\
&= \frac{1}{k}\mathbb{E}\left[tr\left(\textbf{x}^T\Omega^T\Omega\textbf{x}\right)\right]\\
&= \frac{1}{k}\mathbb{E}\left[tr\left(\Omega^T\Omega\textbf{x}\textbf{x}^T\right)\right]\\
&= \frac{1}{k}tr\left(\mathbb{E}\left[\Omega^T\Omega\right]\textbf{x}\textbf{x}^T\right)\\\mathbb{E}[\Omega_{ij}] = 0,\mathbb{E}[\Omega_{ij}^2] = 1 \\
\text{Each element of }\Omega \text{ is independent, thus:}\\
\mathbb{E}[\Omega_{ij}*\Omega_{\neq ij}] = 0\\
&= \frac{1}{k}tr\left(\mathbb{E}\left[k*\textbf{I}_p\right]\textbf{x}\textbf{x}^T\right)\\
&=tr\left(\textbf{x}\textbf{x}^T\right)\\
&=
{\lvert\lvert\textbf{x}\rvert\rvert}^2\\
\end{aligned}
$$

#### Part 2

The only things necessary for part 1 to be true are that the distribution Q has an expected value of 0 and a variance of 1. This makes it such that for any value in $\Omega$, $\mathbb{E}[\Omega_{ij}] = 0,\mathbb{E}[\Omega_{ij}^2] = 1$

### 2.

We solve for  $\text{arg }\underset{\beta}{\text{min}}\mathbb{P}_n(Y - (\Omega X)^{T} \beta)^2$ through taking the derivative w.r.t to $\beta$ and setting to 0 as done in the following steps.

\begin{align}
0 &= \mathbb{P}_n(Y - (\Omega X)^{T} \beta)\Omega X \\
0 &= \mathbb{P}_n(\Omega XY - \Omega X(\Omega X)^{T} \beta) \\
\beta &= (\mathbb{P}_n\Omega X(\Omega X)^{T})^{-1}\mathbb{P}_n\Omega XY \\
      &= (\mathbb{P}_n\Omega X X^{T}\Omega^{T})^{-1}\mathbb{P}_n\Omega XY \\
      &= (\Omega \{\mathbb{P}_nX X^{T}\} \Omega^{T})^{-1}\Omega \mathbb{P}_nXY
\end{align}

Let $\beta^{*,\Omega}_n = (\Omega \{\mathbb{P}_nX X^{T}\} \Omega^{T})^{-1}\Omega \mathbb{P}_nXY$.

Notice that this only depends on the data from $\mathbb{P}_n X X^{T}$ and $\mathbb{P}_nX Y$ so we only need to sweep through the data once to construct this estimator.

We can partition our dataset, $\{X_i, Y_i\}^{n}_{i=1}$ , into $C_1, ..., C_k$ and calculate 

$\Sigma_j = \sum_{i \in C_j} X_i X^{T}_i$ and $\Gamma_j = \sum_{i \in C_j} X_i Y_i$.

Then $\beta^{*,\Omega}_n = (\Omega \left\{\sum^{k}_{i=1}\Sigma_i \right\} \Omega^{T})^{-1}\Omega \left\{\sum^{k}_{i=1}\Gamma_i \right\}$

As $X_i X^{T}_i$ is $pxp$ and $X_i Y_i$ is $px1$, so our storage is $O(p^2 + p) \approx O(p^2)$


Using this we can now define:
$\hat{\beta}^{\Omega}_n = \Omega^{T}\beta^{*,\Omega}_n$ and 

$\hat{\beta}^{\text{ave}}_n = \frac{1}{B} \sum^{B}_{b=1} = \hat{\beta}^{\Omega_{(b)}}_n$ can be evaluated with only one sweep through the data.

### 3.

From problem 2:

We can partition our dataset into $C_1, ..., C_k$ and calculate 

$\Sigma_j = \sum_{i \in C_j} X_i X^{T}_i$ and $\Gamma_j = \sum_{i \in C_j} X_i Y_i$.

Then $\beta^{*,\Omega}_n = (\Omega \left\{\sum^{k}_{i=1}\Sigma_i \right\} \Omega^{T})^{-1}\Omega \left\{\sum^{k}_{i=1}\Gamma_i \right\}$

We can therefore sweep through the data once and then find each $\hat{\beta}_n^{\Omega^{(b)}}$ at the end, the values of which we may use to calculate the final value of $\hat{\beta}_n^{\text{ave}}$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# model parameters
n = 10000
p = 100
j = 50
B = 25
k_min = 1
k_max = 100

# generate sample data
X = np.random.normal(size=(p, n))
β = np.random.uniform(-3, 3, p) 
y = X.T @ β + np.random.normal(p)

# partitions
C = np.arange(n)
np.random.shuffle(C)
C = C.reshape(j, n // j)

# sufficient stats
Σ = np.zeros((p, p))
Γ = np.zeros(p)

for i in range(j):
    Σ += X[:, C[i]] @ X[:, C[i]].T
    Γ += X[:, C[i]] @ y[C[i]]
    
# result storage
β_ave = np.zeros(p)

for _ in range(B):
    k = np.random.randint(low=k_min, high=k_max)             # P(k) is uniform on set
    Ω = np.random.binomial(n=1, p=0.5, size=(k, p)) * 2 - 1  # Rademacher = binomial * 2 - 1
    β_ave += Ω.T @ np.linalg.inv(Ω @ Σ @ Ω.T) @ Ω @ Γ
    
β_ave /= B

plt.scatter(β, β_ave)
plt.title("True β vs. β_ave")
plt.xlabel("β")
plt.ylabel("β_ave")
plt.show()

### 4.

In [40]:
from numba import njit, prange
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as mse

np.random.seed(12345)

# model parameters
n = 500
p = 50
k = 25
s = 1000

# generate sample data
X = np.ascontiguousarray(np.random.normal(size=(p, n)))
β = np.random.uniform(-3, 3, p)
y = X.T @ β + np.random.normal(p)

# generate out-of-sample data
X_test = np.random.normal(size=(p, 100)) 
y_test = X_test.T @ β + np.random.normal(p)

@njit(parallel=True)
def f_n(X, Ω):
    p, n = X.shape
    out = 0
    for i in prange(n):
        for j in prange(n):
             out += (X[:, i].T @ X[:, j] - X[:, i].T @ Ω.T @ Ω @ X[:, j])**2
    return out

min_f = np.Inf
Ω_hat = np.zeros((k, p))

pbar = tqdm(total=s, position=0)
for _ in range(s): 
    Ω = np.ascontiguousarray(np.random.normal(size=(k, p)))
    f = f_n(X, Ω)
    if f < min_f:
        min_f = f
        Ω_hat = Ω
    pbar.update()

# k-dim embedding
β_Ω = Ω.T @ np.linalg.inv(Ω @ X @ X.T @ Ω.T) @ Ω @ X @ y
y_n = X_test.T @ β_Ω
mse_k = mse(y_test, y_n)

# random normal
Ω_r = np.random.normal(size=(k, p))
β_r = Ω_r.T @ np.linalg.inv(Ω_r @ X @ X.T @ Ω_r.T) @ Ω_r @ X @ y
y_r = X_test.T @ β_r
mse_r = mse(y_test, y_r)

100%|██████████| 1000/1000 [02:46<00:00,  5.89it/s]

In [41]:
print(
    'Optimal Ω OOS MSE:', mse_k, '\n',
    'Random Ω OOS MSE:', mse_r
)

Optimal Ω OOS MSE: 2370.4521847519 
 Random Ω OOS MSE: 2518.5906209531213
